# ---Day 9: Sensor Boost---

## ---Part One---

You've just said goodbye to the rebooted rover and left Mars when you receive a faint distress signal coming from the asteroid belt. It must be the Ceres monitoring station!

In order to lock on to the signal, you'll need to boost your sensors. The Elves send up the latest ***BOOST*** program - Basic Operation Of System Test.

While BOOST (your puzzle input) is capable of boosting your sensors, for tenuous safety reasons, it refuses to do so until the computer it runs on passes some checks to demonstrate it is a ***complete Intcode computer***.

Your existing Intcode computer (day5) is missing one key feature: it needs support for parameters in ***relative mode***.

Parameters in mode 2, ***relative mode***, behave very similarly to parameters in ***position mode***: 
> The parameter is interpreted as a position. Like position mode, parameters in relative mode can be read from or written to.

The important difference is that relative mode parameters don't count from address `0`. Instead, they count from a value called the ***relative base***. The ***relative base*** starts at `0`.

The address a relative mode parameter refers to is itself ***plus*** the current ***relative base***. When the relative base is `0`, relative mode parameters and position mode parameters with the same value refer to the same address.

For example, given a relative base of `50`, a relative mode parameter of `-7` refers to memory address `50 + -7 = 43`.

The relative base is modified with the ***relative base offset*** instruction:

* Opcode 9 ***adjusts the relative base*** by the value of its only parameter. The relative base increases (or decreases, if the value is negative) by the value of the parameter.

**For example, if the relative base is `2000`, then after the instruction `109,19`, the relative base would be `2019`. If the next instruction were `204,-34`, then the value at address `1985` would be output.**

Your Intcode computer will also need a few other capabilities:

* The computer's available memory should be much larger than the initial program. Memory beyond the initial program starts with the value 0 and can be read or written like any other memory. (It is invalid to try to access memory at a negative address, though.)
* The computer should have support for large numbers. Some instructions near the beginning of the BOOST program will verify this capability.

Here are some example programs that use these features:

* `109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99` takes no input and produces a <a src="https://en.wikipedia.org/wiki/Quine_(computing)">copy of itself</a> as output.
* `1102,34915192,34915192,7,4,7,99,0` should output a 16-digit number.
* `104,1125899906842624,99` should output the large number in the middle.

The BOOST program will ask for a single input; run it in test mode by providing it the value 1. It will perform a series of checks on each opcode, output any opcodes (and the associated parameter modes) that seem to be functioning incorrectly, and finally output a BOOST keycode.

Once your Intcode computer is fully functional, the BOOST program should report no malfunctioning opcodes when run in test mode; it should only output a single value, the BOOST keycode. 

**What BOOST keycode does it produce?**

### Computer class from day07 with additional day09 requirements

**Things to possibly look into:**
>* Why do I have to write over the parameters that an instruction writes for opcode 1, 2, 7, and 8
>* Deciding on how much memory to add to the program on initialization

In [1]:
class Computer:
    def __init__(self, program, input_val=1):
        self.program = program + [0 for i in range(1000)]
        self.input = input_val
        self.pointer = 0
        self.instruction = 0
        self.base = 0
        self.out = 0
        self.opcode = { 1 : self.add,
                       2 : self.multiply,
                       3 : self.save,
                       4 : self.output,
                       5 : self.jump_true,
                       6 : self.jump_false,
                       7 : self.less_than,
                       8 : self.is_equal,
                       9 : self.change_base,
                       99 : self.halt }
    
    def set_instruction(self):
        self.instruction = self.program[self.pointer]
        self.pointer += 1
        
    def get_opcode(self):
        return self.instruction % 100
    
    def get_modes(self):
        return self.instruction // 100

    def get_parameters(self, modes, num_parameters=3):
        mode = [ (modes // 10**n % 10) for n in range(num_parameters) ]
        parameters = []
        for i in range(self.pointer, self.pointer+num_parameters):
            if mode[i-self.pointer]:
                if mode[i-self.pointer] == 2:
                    parameters.append(self.program[self.program[i]+self.base])
                else:
                    parameters.append(self.program[i])
            else:
                parameters.append(self.program[self.program[i]])
        return parameters
    
    def set_write(self, modes, num_parameters):
        relative = [ (modes // 10**n % 10) for n in range(num_parameters) ][-1]
        if relative:
            return self.program[self.pointer+num_parameters-1]+self.base
        return self.program[self.pointer+num_parameters-1]

    def add(self, step=3):
        values = self.get_parameters(self.get_modes(), step)
        # Write over last parameter because it should always be a value indicating the location
        values[-1] = self.set_write(self.get_modes(), step)
        self.program[values[-1]] = values[0] + values[1]
        self.pointer += step
        
    def multiply(self, step=3):
        values = self.get_parameters(self.get_modes(), step)
        # Write over last parameter because it should always be a value indicating the location
        values[-1] = self.set_write(self.get_modes(), step)
        self.program[values[-1]] = values[0] * values[1]
        self.pointer += step
        
    def save(self, step=1):
        if self.get_modes():
            self.program[self.program[self.pointer]+self.base] = self.input
        else:
            self.program[self.program[self.pointer]] = self.input
        self.pointer += step
        
    def output(self, step=1):
        self.out = self.get_parameters(self.get_modes(), step)[0]
        print(f'Output: {self.out}')
        self.pointer += step 
        
    def jump_true(self, step=2):
        values = self.get_parameters(self.get_modes(), step)
        if values[0] != 0:
            self.pointer = values[1]
        else:
            self.pointer += step
            
    def jump_false(self, step=2):
        values = self.get_parameters(self.get_modes(), step)
        if values[0] == 0:
            self.pointer = values[1]
        else:
            self.pointer += step
            
    def less_than(self, step=3):
        values = self.get_parameters(self.get_modes(), step)
        # Write over last parameter because it should always be a value indicating the location
        values[-1] = self.set_write(self.get_modes(), step)
        self.program[values[-1]] = int(values[0] < values[1])
        self.pointer += step
    
    def is_equal(self, step=3):
        values = self.get_parameters(self.get_modes(), step)
        # Write over last parameter because it should always be a value indicating the location
        values[-1] = self.set_write(self.get_modes(), step)
        self.program[values[-1]] = values[0] * values[1]
        self.program[values[-1]] = int(values[0]==values[1])
        self.pointer += step
    
    def change_base(self, step=1):
        value = self.get_parameters(self.get_modes(), step)
        self.base += value[0]
        self.pointer += step
        
    def halt(self):
        print("Program Halted.")
        self.instruction = -1
    
    def run(self):
        while self.instruction != -1:
            #print(self.program, self.base)
            self.set_instruction()
            #print(self.pointer, self.instruction) # Used for Testing
            self.opcode[self.get_opcode()]()
            # print(self.program[:self.pointer+1], self.program[self.pointer:self.pointer+4]) # Used for Testing

#### Examples for new functionality

In [2]:
ex0 = [109,2019,204,-2014,99,42]
ex0_cpu = Computer(ex0)
ex0_cpu.run()

Output: 42
Program Halted.


In [3]:
#takes no input and produces a copy of itself as output
ex1 = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
ex1_cpu = Computer(ex1)
ex1_cpu.run()

Output: 109
Output: 1
Output: 204
Output: -1
Output: 1001
Output: 100
Output: 1
Output: 100
Output: 1008
Output: 100
Output: 16
Output: 101
Output: 1006
Output: 101
Output: 0
Output: 99
Program Halted.


In [4]:
#should output a 16-digit number.
ex2 = [1102,34915192,34915192,7,4,7,99,0]
ex2_cpu = Computer(ex2)
ex2_cpu.run()

Output: 1219070632396864
Program Halted.


In [5]:
#should output the large number in the middle.
ex3 = [104,1125899906842624,99]
ex3_cpu = Computer(ex3)
ex3_cpu.run()

Output: 1125899906842624
Program Halted.


In [6]:
with open('day9.txt') as f:
    boost = [ int(i) for i in f.read().split(',') ]
    boost_cpu = Computer(boost, 1)
    boost_cpu.run()

Output: 3063082071
Program Halted.


## ---Part Two---

***You now have a complete Intcode computer.***

Finally, you can lock on to the Ceres distress signal! You just need to boost your sensors using the BOOST program.

The program runs in sensor boost mode by providing the input instruction the value 2. Once run, it will boost the sensors automatically, but it might take a few seconds to complete the operation on slower hardware. In sensor boost mode, the program will output a single value: ***the coordinates of the distress signal***.

Run the BOOST program in sensor boost mode. **What are the coordinates of the distress signal?**

In [7]:
with open('day9.txt') as f:
    sensor_boost = [ int(i) for i in f.read().split(',') ]
    sensor_boost_cpu = Computer(boost, 2)
    sensor_boost_cpu.run()

Output: 81348
Program Halted.
